In [2]:
%pip install -U marshmallow

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dataclasses import dataclass, field
import datetime as dt
from marshmallow import Schema, fields

@dataclass
class User:
    name: str
    age: int
    email: str
    created_at: dt.datetime = field(default_factory=dt.datetime.now)


class UserSchema(Schema):
    name = fields.Str()
    age = fields.Int()
    email = fields.Email()
    created_at = fields.DateTime()

UserSchemaTwo = Schema.from_dict({
    "name": fields.Str(),
    "age": fields.Int(),
    "email": fields.Email(),
    "created_at": fields.DateTime()
})

from pprint import pprint

user = User(name="John Doe", age=30, email="some@example.com")
user_schema = UserSchema()
res = user_schema.dump(user)
pprint(res, indent=2)

{ 'age': 30,
  'created_at': '2025-05-01T21:26:12.908794',
  'email': 'some@example.com',
  'name': 'John Doe'}


In [4]:
res_2 = user_schema.dumps(user)
pprint(res_2, indent=2)

('{"name": "John Doe", "age": 30, "email": "some@example.com", "created_at": '
 '"2025-05-01T21:26:12.908794"}')


In [6]:
only_name_schema =UserSchema(only=["name"])
only_name = only_name_schema.dump(user)
pprint(only_name, indent=2)

{'name': 'John Doe'}


In [7]:
all_fields_schema = UserSchema(exclude=["email"])
all_fields = all_fields_schema.dump(user)
pprint(all_fields, indent=2)

{'age': 30, 'created_at': '2025-05-01T21:26:12.908794', 'name': 'John Doe'}


In [8]:
des_user = user_schema.load(res)
pprint(des_user, indent=2)

{ 'age': 30,
  'created_at': datetime.datetime(2025, 5, 1, 21, 26, 12, 908794),
  'email': 'some@example.com',
  'name': 'John Doe'}


In [12]:
from marshmallow import post_load

class UserSchemaWithPostLoad(Schema):
    name = fields.Str()
    age = fields.Int()
    email = fields.Email()
    created_at = fields.DateTime()

    @post_load
    def make_user(self, data, **kwargs):
        return User(**data)

user_ = UserSchemaWithPostLoad()
res_ = user_.load(res)
pprint(res_, indent=2)

User(name='John Doe',
     age=30,
     email='some@example.com',
     created_at=datetime.datetime(2025, 5, 1, 21, 26, 12, 908794))


In [13]:
users = [
    User(name="Adam Snow", age=23, email="adam@example.com"),
    User(name="John Doe", age=33, email="john@example.com"),
    User(name="Grey Shepherd", age=22, email="grey@example.com"),
]
many_schema = UserSchema(many=True)
users_ = many_schema.dump(users)
pprint(users_, indent=2)

[ { 'age': 23,
    'created_at': '2025-05-01T21:41:56.064091',
    'email': 'adam@example.com',
    'name': 'Adam Snow'},
  { 'age': 33,
    'created_at': '2025-05-01T21:41:56.064478',
    'email': 'john@example.com',
    'name': 'John Doe'},
  { 'age': 22,
    'created_at': '2025-05-01T21:41:56.064480',
    'email': 'grey@example.com',
    'name': 'Grey Shepherd'}]


In [16]:
import marshmallow
data_with_invalid = [
    {"email": "mick@stones.com", "name": "Mick"},
    {"email": "invalid", "name": "Invalid"},  # invalid email
    {"email": "keith@stones.com", "name": "Keith"},
    {"email": "charlie@stones.com"},  # missing "name"
]
try:
    UserSchema(many=True).load(data_with_invalid)
except marshmallow.ValidationError as e:
    pprint(e.messages, indent=2)



{1: {'email': ['Not a valid email address.']}}


In [19]:
class AnimalSchema(Schema):
    name = fields.Str(validate=marshmallow.validate.Length(max=55))
    ability = fields.Str(validate=marshmallow.validate.OneOf(
        ["fly", "swim", "run", "walk"]
    ))
    age = fields.Int(validate=marshmallow.validate.Range(min=0, max=40))
    weight = fields.Float(validate=marshmallow.validate.Range(min=0, max=100))
    height = fields.Float(validate=marshmallow.validate.Range(min=0, max=3))

try:
    AnimalSchema().load({
        "name": "Elephant",
        "ability": "swim",
        "age": 10,
        "weight": 5000,
        "height": 3.5
    })
except marshmallow.ValidationError as e:
    pprint(e.messages, indent=2)


{ 'height': ['Must be greater than or equal to 0 and less than or equal to 3.'],
  'weight': [ 'Must be greater than or equal to 0 and less than or equal to '
              '100.']}


In [ ]:
def validate_momoxian(value):
    if not value.endswith("@momox.biz"):
        raise marshmallow.ValidationError("Email must end with @momox.biz")
    return value

import uuid

class MomoxSchema(UserSchema):
    id_ = fields.UUID(load_default=uuid.uuid1)
    email = fields.Email(validate=validate_momoxian)
    age = fields.Int(required=True, error_messages={"required": "Age is required."})

    class Meta:
        unknown = marshmallow.RAISE

    @marshmallow.validates("name")
    def validate_name(self, value: str, data_key:  str):
        if len(value) < 3:
            raise marshmallow.ValidationError(f"Too short with {len(value)} chars")

momox_user = MomoxSchema(unknown=marshmallow.EXCLUDE)
try:
    pprint(
        momox_user.load({
            "name": "Joe",
            "age": 43,
            "email": "epastor@momox.biz",
            "modality": True
        }, unknown=marshmallow.INCLUDE), # overrides `unknown` value in Meta and instantiation
        indent=2
    )
except marshmallow.ValidationError as e:
    pprint(e.messages, indent=2)

{ 'age': 43,
  'email': 'epastor@momox.biz',
  'id_': UUID('84199f0c-26d0-11f0-8a59-825c702cfe38'),
  'modality': True,
  'name': 'Joe'}


In [41]:
if error := MomoxSchema().validate({"name": "Joe", "email": "e.pastor@momox.biz", "age": 30}):
    pprint(f"error: {error}")


In [44]:
class FlightSchema(Schema):
    from_ = fields.Str(data_key="Source")
    to_ = fields.Str(data_key="Destination")

c = FlightSchema().dump({"from_": "NYC", "to_": "BER"})
FlightSchema().load(c)

{'from_': 'NYC', 'to_': 'BER'}

In [47]:
@dataclass
class Blog:
    title: str
    author: User

class BlogSchema(Schema):
    title = fields.String()
    author = fields.Nested(MomoxSchema(only=("email",)))

user = User(name="Alex", age=23, email="alex@mail.com")
blog = Blog(title="Something Different", author=user)

res = BlogSchema().dump(blog)
pprint(res)

{'author': {'email': 'alex@mail.com'}, 'title': 'Something Different'}


In [48]:
class CustomPinCodeField(fields.Field[list[int]]):

    def _serialize(self, value, attr, obj, **kwargs):
        if value is None:
            return ""
        return "-".join(str(i) for i in value)

    def _deserialize(self, value, attr, data, **kwargs):
        if not value:
            return []
        try:
            return [int(i) for i in value.split("-")]
        except ValueError as e:
            raise marshmallow.ValidationError("Invalid pin code format") from e
    
class UserWithPinCodeSchema(MomoxSchema):
    pincode = CustomPinCodeField()
    since_created = fields.Method("get_since_created")
    employee_id = fields.Function(lambda obj: f"MX-{obj.email.split('@')[0]}")

    def get_since_created(self, obj):
        return dt.datetime.now().day - obj.created_at.day

user = User(name="John Doe", age=30, email="clay@momox.biz")
user.pincode = [1, 2, 3, 4]

res = UserWithPinCodeSchema().dump(user)
pprint(res, indent=2)


{ 'age': 30,
  'created_at': '2025-05-02T00:35:29.481636',
  'email': 'clay@momox.biz',
  'employee_id': 'MX-clay',
  'name': 'John Doe',
  'pincode': '1-2-3-4',
  'since_created': 0}
